# Сборка семантического графа

#### Импорт библиотек

In [1]:
import geopandas as gpd
import networkx as nx
from pathlib import Path

from sloyka import Semgraph

package_dir = str(Path.cwd().parent)

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Инициализация класса семантического графа

In [2]:
sm = Semgraph()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Загрузка данных
###### Рекомендуется использовать после класса Geocoder

In [3]:
data_path = package_dir + "/sloyka/sample_data/processed/"

In [4]:
test_gdf  = gpd.read_file(data_path + "-10696.geojson")

#### Построение семантического графа

In [5]:
G = sm.build_graph(test_gdf[:len(test_gdf)//2],
                   id_column='id',
                   text_column='text',
                   text_type_column='type',
                   toponym_column='only_full_street_name_numbers',
                   toponym_name_column='initial_street',
                   toponym_type_column='Toponims',
                   post_id_column='post_id',
                   parents_stack_column='parents_stack',
                   location_column='Location',
                   geometry_column='geometry')

Extracting keywords from post chains...


100%|██████████| 181/181 [00:38<00:00,  4.68it/s]


Extracting keywords from comment chains...


100%|██████████| 25/25 [00:01<00:00, 21.48it/s]


Extracting keywords from replies...


100%|██████████| 19/19 [00:01<00:00, 18.88it/s]


Calculating semantic closeness...


100%|██████████| 13861/13861 [00:17<00:00, 805.47it/s]


###### Получаем количество узлов графа

In [6]:
len(G.nodes)

233

###### Обновляем граф

In [7]:
G = sm.update_graph(G,
                    test_gdf[len(test_gdf)//2:],
                    id_column='id',
                    text_column='text',
                    text_type_column='type',
                    toponym_column='only_full_street_name_numbers',
                    toponym_name_column='initial_street',
                    toponym_type_column='Toponims',
                    post_id_column='post_id',
                    parents_stack_column='parents_stack',
                    location_column='Location',
                    geometry_column='geometry')

Extracting keywords from post chains...


0it [00:00, ?it/s]


Extracting keywords from comment chains...


100%|██████████| 30/30 [00:02<00:00, 14.63it/s]


Extracting keywords from replies...


100%|██████████| 16/16 [00:00<00:00, 22.02it/s]


Calculating semantic closeness...


100%|██████████| 153/153 [00:00<00:00, 825.91it/s]


###### Проверяем что количество узлов стало больше

In [8]:
len(G.nodes)

262

###### Смотрим атрибуты узла с номером дома в id узла

In [9]:
G.nodes['набережная канала Грибоедова 108']

{'tag': 'TOPONYM',
 'counts': 1,
 'Location': 'набережная канала Грибоедова, Апраксин двор, округ № 78, Санкт-Петербург, Северо-Западный федеральный округ, 191186, Россия',
 'Lat': 30.3265614,
 'Lon': 59.9351265,
 'text_ids': '231911'}

###### Добавляем в атрибуты узлов со зданиями атрибут с текстом, из которого был извлечён топоним

In [10]:
G = sm.get_house_text_id(G, test_gdf, 'id', 'text')

###### Проверяем, что атрибут добавился

In [11]:
G.nodes['набережная канала Грибоедова 108']

{'tag': 'TOPONYM',
 'counts': 1,
 'Location': 'набережная канала Грибоедова, Апраксин двор, округ № 78, Санкт-Петербург, Северо-Западный федеральный округ, 191186, Россия',
 'Lat': 30.3265614,
 'Lon': 59.9351265,
 'text_ids': '231911',
 'extracted_from': 'Боже..108!!'}

###### Смотрим количество связей в графе

In [12]:
len(G.edges)

267

###### Добавляем связи расстояния между топонимами в граф

In [13]:
G = sm.calculate_distances(G)

100%|██████████| 3081/3081 [00:00<00:00, 11555.35it/s]


###### Проверяем, что в граф добавились новые связи

In [14]:
len(G.edges)

6429

###### Загружаем слои территориально-административного деления города

In [15]:
districts = gpd.read_file(data_path + "districts.geojson")
mo = gpd.read_file(data_path + "municipalities.geojson")

###### Смотрим количество узлов в графе

In [16]:
len(G.nodes)

262

###### Добавляем узлы районов и муниципалитетов в граф

In [17]:
G = sm.add_city_graph(G, districts, mo, 'city_id', 'district_name', 'name', 'geometry')

###### Проверяем, что в графе появились новые узлы

In [18]:
len(G.nodes)

392

###### Сохранение полученного графа в формате ``graphml``

In [ ]:
# nx.write_graphml(G, "semantic_graph_example.graphml", encoding = 'utf-8')